In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score

%load_ext nim_magic

## Uniform Call Syntax

In [2]:
%%inim

let
  bobDaly: string = "bob"
        
echo "camelcase: " & bobDaly
echo "snake_case: " & bob_daly
echo "all lower: " & bobdaly

camelcase: bob
snake_case: bob
all lower: bob


## Fibonacci Examples

### Example 1

In [3]:
# python implementation
def fib1(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib1(n - 2) + fib1(n - 1)

In [4]:
%%nimpy --opt:speed

proc nim_fib1(n: int64): int64 {.exportpy} =
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return nim_fib1(n - 2) + nim_fib1(n - 1)

In [5]:
fib1(35)   # python

9227465

In [6]:
nim_fib1(35)   # nim

9227465

In [7]:
%%timeit   # python
fib1(35)

5.1 s ± 25.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit   # nim
nim_fib1(35)

280 ms ± 3.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%inim --opt:speed
    
proc nim_fib1(n: int64): int64 = # functions contain an implicit 'result' that negates the need for return
    if n == 0:
        result = 0
    elif n == 1:
        result = 1
    else:
        result = nim_fib1(n - 2) + nim_fib1(n - 1)
        
echo nim_fib1(35)

9227465


### Example 2

In [10]:
# python implementation
def fib2(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        last, current = 0, 1
        for i in range(2, n + 1):
            value = last + current
            last, current = current, value
        return value

In [11]:
%%nimpy --opt:speed

# you can see scoping come into play here
proc nim_fib2(n: int64): int64 {.exportpy.} =
    var   # define variables
      last, current: int64
            
    if n == 0:
        result = 0
    elif n == 1:
        result = 1
    else:
        last = 0
        current = 1
        for i in 2..n:   # if I wanted non-includsive, use 2..<n
            result = last + current
            last = current
            current = result

In [12]:
fib2(75)

2111485077978050

In [13]:
nim_fib2(75)

2111485077978050

In [14]:
%%timeit
fib2(75)

6.86 µs ± 63.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
%%timeit
nim_fib2(75)

442 ns ± 20.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Working with numpy

### Read from an array, return an array

In [16]:
%%nimpy -d:release -d:danger --opt:speed --passC:-ffast-math
                
import nimpy/raw_buffers

type
    T = float64
    
let np = pyImport("numpy")   # import your Python implementation of numpy here

proc multiply_array_by_2(npArr: PyObject): PyObject {.exportpy.} =
    var npArr: PyObject = np.asarray(npArr, order="C")
        
    # check the type expected
    if $npArr.dtype != "float64":
        raise newException(ValueError, "numpy dtype should be of type float64!")
        
    var
        a = np.empty(shape=npArr.shape, dtype="f8", order="C")
        n = a.size.to(int)
        npArrBuf, aBuf: RawPyBuffer   # from nimpy
    npArr.getBuffer(npArrBuf, PyBUF_WRITABLE or PyBUF_ND)   # from nimpy - we are accessing the passed array here
    a.getBuffer(aBuf, PyBUF_WRITABLE or PyBUF_ND)   
    for i in 0..<n:
        var
            step = uint(i * sizeof(T))
            aMem = cast[uint](aBuf.buf) + step
            npArrMem = cast[uint](npArrBuf) + step
        var value = 2.0 * cast[ptr T](npArrMem)[]
        cast[ptr T](aMem)[] = value
    aBuf.release()      # need to free memory
    npArrBuf.release()
    return a

In [17]:
arr = np.arange(100).reshape((10, 10)).astype(np.float64)

In [18]:
2.0 * arr

array([[  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.],
       [ 20.,  22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.],
       [ 40.,  42.,  44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.],
       [ 60.,  62.,  64.,  66.,  68.,  70.,  72.,  74.,  76.,  78.],
       [ 80.,  82.,  84.,  86.,  88.,  90.,  92.,  94.,  96.,  98.],
       [100., 102., 104., 106., 108., 110., 112., 114., 116., 118.],
       [120., 122., 124., 126., 128., 130., 132., 134., 136., 138.],
       [140., 142., 144., 146., 148., 150., 152., 154., 156., 158.],
       [160., 162., 164., 166., 168., 170., 172., 174., 176., 178.],
       [180., 182., 184., 186., 188., 190., 192., 194., 196., 198.]])

In [19]:
multiply_array_by_2(arr)

array([[  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.],
       [ 20.,  22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.],
       [ 40.,  42.,  44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.],
       [ 60.,  62.,  64.,  66.,  68.,  70.,  72.,  74.,  76.,  78.],
       [ 80.,  82.,  84.,  86.,  88.,  90.,  92.,  94.,  96.,  98.],
       [100., 102., 104., 106., 108., 110., 112., 114., 116., 118.],
       [120., 122., 124., 126., 128., 130., 132., 134., 136., 138.],
       [140., 142., 144., 146., 148., 150., 152., 154., 156., 158.],
       [160., 162., 164., 166., 168., 170., 172., 174., 176., 178.],
       [180., 182., 184., 186., 188., 190., 192., 194., 196., 198.]])

In [20]:
%%timeit
2.0 * arr

1.34 µs ± 24.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
%%timeit
multiply_array_by_2(arr)

13.4 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Somers' D Loss Approximation Example

In [22]:
np.random.seed(19)
yp = np.random.uniform(size=10000).astype(np.float64)
yt = np.random.binomial(1, yp).astype(np.float64)

print("Somers' D: {:.5f}".format(2.0 * roc_auc_score(yt, yp) - 1.0))

Somers' D: 0.65432


In [23]:
def sd_approx(yt, yp, alpha=1.0, m=10):
    """Calculate a continuous function approximation of Somers' D."""
    
    # separate positive and negative values
    pind = np.nonzero(yt == 1)[0]
    pvec = yp[pind]
    nind = np.nonzero(yt == 0)[0]
    nvec = yp[nind]
    
    # loop to get answer
    k = 2.0 * alpha
    pvec = pvec ** k   # exponentiation only once
    nvec = nvec ** k
    loss = 0.0
    i = 0
    while i < pvec.shape[0]:
        pvec_sub = pvec[i: i + m]
        pind_sub = pind[i: i + m]
        mat = -pvec_sub.reshape((-1, 1)) + nvec.reshape((1, -1))
        mat /= (pvec_sub.reshape((-1, 1)) + nvec.reshape((1, - 1)))
        loss += mat.sum()
        i += m
    return -loss / (pvec.shape[0] * nvec.shape[0])

In [24]:
sd_approx(yt, yp, alpha=30.0, m=1)

0.6537452577678036

In [25]:
%%timeit
sd_approx(yt, yp, alpha=30.0, m=1)

203 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%nimpy -d:release -d:danger --opt:speed --passC:-ffast-math

import math

# operator overloading examples with generics (arr-like ptr indexing)
proc `[]`[T](p: ptr T, i: int): T =
    ## Access a pointer location directly with indexing
    let
        size_t: uint = uint(sizeof(T))
        mem_location: uint = cast[uint](p) + uint(i) * size_t
    result = cast[ptr T](mem_location)[]
    
proc `[]=`[T](p: ptr T, i: int, val: T) =
    ## Assign to a pointer location directly with indexing
    let
        size_t: uint = uint(sizeof(T))
        mem_location: uint = cast[uint](p) + uint(i) * size_t
    cast[ptr T](mem_location)[] = val
    
proc nim_sd_approx(yt, yp: PyObject, alpha: float64 = 1.0): float64 {.exportpy.} =
    ## Approximate the numerator of Somers' D using continuous functions.
    ##
    ## The value is approximate in the sense that scores that are close to one another
    ## can be 'partially' concordant or discordant. As alpha increases, the
    ## approximation improves.
    
    # check that types are float64
    if not ($yt.dtype == "float64" and $yp.dtype == "float64"):
        raise newException(ValueError, "numpy array dtypes must be float64")
        
    # initialize buffers and buffer size
    var
        ytBuf: ptr float64 = cast[ptr float64](yt.ctypes.data.to(uint))
        ypBuf: ptr float64 = cast[ptr float64](yp.ctypes.data.to(uint))
        n: int = yt.size.to(int)   # should check that the sizes match here for safety
        pind: seq[int] = newSeq[int]()   # to hold positive indices
        nind: seq[int] = newSeq[int]()   # to hold negative indices
            
    # separate negative and positive instances
    for idx in 0..<n:
        if ytBuf[idx] == 1.0:
            pind.add(idx)
        else:
            nind.add(idx)
            
    # run calculations
    let ke: float = 2.0 * alpha
    var ypExp: seq[float64] = newSeq[float64](n)
    
    # perform exponent operations only once
    for idx in 0..<n:
        ypExp[idx] = pow(ypBuf[idx], ke)
        
    # loop and calculate
    result = 0.0
    for p_idx in items(pind):   # items is an example of an iterator
        var pi: float64 = ypExp[p_idx]
        for n_idx in items(nind):
            var pj: float = ypExp[n_idx]
            result += (pj - pi) / (pj + pi)
            
    # convert to somers' d
    result = -result / float64(len(pind) * len(nind))

In [27]:
nim_sd_approx(yt, yp, alpha=30.0)

0.6537452577699266

In [28]:
%%timeit
nim_sd_approx(yt, yp, alpha=30.0)

47.2 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%nim_clear

Removed temporary files used by nim_magic.
